In [ ]:
# Colab에서 keras-tuner 설치
!pip install keras-tuner

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch
import numpy as np

# 하이퍼파라미터 튜닝을 위한 모델 빌더 함수
def build_model(hp):
    model = keras.Sequential()

    # 1. Conv2D 필터 수 튜닝 (32-128)
    filters = hp.Int('conv_1_filters', min_value=32, max_value=128, step=32)
    model.add(layers.Conv2D(
        filters=filters,
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),  # 2. kernel size 튜닝
        activation='relu',
        input_shape=(32, 32, 3)
    ))

    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    # 3. Dropout rate 튜닝 (0.1-0.5)
    dropout_rate = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Conv2D(64, 3, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())

    # 4. Dense 레이어의 유닛 수 튜닝 (64-256)
    dense_units = hp.Int('dense_units', min_value=64, max_value=256, step=64)
    model.add(layers.Dense(dense_units, activation='relu'))

    model.add(layers.Dense(10, activation='softmax'))

    # 컴파일
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# GPU 확인
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 검증 세트 분리
val_size = 5000
x_val = x_train[-val_size:]
y_val = y_train[-val_size:]
x_train = x_train[:-val_size]
y_train = y_train[:-val_size]

# 하이퍼파라미터 튜닝 실행
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # 시도 횟수
    directory='cifar10_tuning',
    project_name='cifar10_cnn'
)

tuner.search(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_val, y_val),
    batch_size=64
)

# 최적의 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\n최적의 하이퍼파라미터:")
print(f"Conv2D 필터 수: {best_hps.get('conv_1_filters')}")
print(f"Conv2D kernel size: {best_hps.get('conv_1_kernel')}")
print(f"Dropout rate: {best_hps.get('dropout_1')}")
print(f"Dense 유닛 수: {best_hps.get('dense_units')}")

# 최적의 모델로 최종 평가
best_model = tuner.get_best_models(num_models=1)[0]
evaluation = best_model.evaluate(x_test, y_test)
print(f"\n테스트 세트 성능:")
print(f"Loss: {evaluation[0]:.4f}")
print(f"Accuracy: {evaluation[1]:.4f}")

Trial 5 Complete [00h 01m 02s]
val_accuracy: 0.7305999994277954

Best val_accuracy So Far: 0.7305999994277954
Total elapsed time: 00h 04m 16s

최적의 하이퍼파라미터:
Conv2D 필터 수: 128
Conv2D kernel size: 3
Dropout rate: 0.1
Dense 유닛 수: 128


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7184 - loss: 0.9840

테스트 세트 성능:
Loss: 0.9963
Accuracy: 0.7142



최적화 결과
	•	검증 정확도(val_accuracy)가 약 73.06%로 나왔으며, 이는 지금까지의 시도 중 가장 좋은 결과입니다.
	•	총 소요 시간은 4분 16초입니다.
최적의 하이퍼파라미터
모델이 가장 좋은 성능을 보인 설정은:
	•	Conv2D 레이어의 필터 수: 128개
	•	컨볼루션 커널 크기: 3x3
	•	드롭아웃 비율: 0.1 (즉, 뉴런의 10%를 무작위로 비활성화)
	•	Dense 레이어의 유닛 수: 128개
최종 테스트 결과
	•	손실(Loss): 0.9963
	•	정확도(Accuracy): 71.42%
이는 모델이 테스트 데이터에 대해 약 71%의 정확도를 보여주며, 실제 현장에서 적용했을 때 10개 중 7개 정도를 올바르게 예측할 수 있다는 의미입니다. 검증 정확도(73.06%)와 테스트 정확도(71.42%)가 비슷한 수준을 보이는 것으로 보아, 모델이 적절하게 일반화되었다고 볼 수 있습니다